In [4]:
import pygame
import random
import time

# Pygame setup
pygame.init()
WIDTH, HEIGHT = 3200, 600
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Task Scheduler Visualization")
FONT = pygame.font.SysFont("Arial", 24)

# Task definition: [Task ID, Duration, Deadline, Priority]
TASKS = [
    (1, 2, 5, 10),
    (2, 1, 3, 5),
    (3, 2, 6, 8),
    (4, 3, 7, 7),
    (5, 1, 4, 6),
    (6, 2, 5, 9),
    (7, 1, 3, 4),
    (8, 3, 7, 6),
    (9, 2, 6, 8),
    (10, 1, 4, 5),
    (11, 2, 5, 7),
    (12, 1, 3, 3),
    (13, 3, 7, 9),
    (14, 2, 6, 10),
    (15, 1, 4, 6),
    (16, 3, 7, 8),
    (17, 2, 5, 9),
    (18, 1, 3, 4),
    (19, 3, 7, 7),
    (20, 2, 6, 8),
]

GENERATIONS = 100
MUTATION_RATE = 0.5
POPULATION_SIZE = 20

# Fitness function
def fitness(schedule):
    current_time = 0
    total_fitness = 0
    
    for task_id in schedule:
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        current_time += duration
        if current_time <= deadline:
            total_fitness += priority
        else:
            total_fitness -= priority # (current_time - deadline) # Penalize missed deadlines
    return total_fitness

# Generate a random individual (schedule)
def create_individual():
    task_ids = [task[0] for task in TASKS]
    random.shuffle(task_ids)
    return task_ids

# Generate initial population
def create_population(size):
    return [create_individual() for _ in range(size)]

# Selection: Choose top individuals based on fitness
def select(population):
    population.sort(key=fitness, reverse=True)
    return population[:len(population) // 2]

# Crossover: Combine two parents to produce offspring
def crossover(parent1, parent2):
    split = len(parent1) // 2
    child = parent1[:split] + [task for task in parent2 if task not in parent1[:split]]
    return child

# Mutation: Randomly swap two tasks in the schedule
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Visualization of the schedule
def draw_schedule(schedule, generation, best_fitness):
    SCREEN.fill((255, 255, 255))  # Clear screen with white background

    # Draw tasks as rectangles on the timeline
    current_time = 0
    for task_id in schedule:
        duration, _, _ = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        x = current_time * 100  # Scale time by 100 pixels
        y = HEIGHT // 2 - 20
        width = duration * 100  # Task duration width
        pygame.draw.rect(SCREEN, (0, 128, 255), (x, y, width, 40))
        task_text = FONT.render(f"T{task_id}", True, (255, 255, 255))
        SCREEN.blit(task_text, (x + 10, y + 10))
        current_time += duration

    # Draw time axis
    pygame.draw.line(SCREEN, (0, 0, 0), (0, HEIGHT // 2 + 30), (WIDTH, HEIGHT // 2 + 30), 2)

    # Display generation and best fitness
    gen_text = FONT.render(f"Generation: {generation}", True, (0, 0, 0))
    fit_text = FONT.render(f"Best Fitness: {best_fitness}", True, (0, 0, 0))
    SCREEN.blit(gen_text, (10, 10))
    SCREEN.blit(fit_text, (10, 40))

    pygame.display.flip()  # Update the display


# Genetic Algorithm with Visualization
def genetic_algorithm():
    population = create_population(POPULATION_SIZE)
    best_schedule = None

    for generation in range(GENERATIONS):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        # Selection
        selected = select(population)

        # Crossover and Mutation
        next_generation = []
        while len(next_generation) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected, 2)
            offspring = crossover(parent1, parent2)
            offspring = mutate(offspring)
            next_generation.append(offspring)

        population = next_generation
        best_schedule = max(population, key=fitness)
        best_fitness = fitness(best_schedule)

        # Draw the schedule
        draw_schedule(best_schedule, generation + 1, best_fitness)

        # Pause briefly to simulate real-time update
        time.sleep(0.5)

    return best_schedule

# Run the Genetic Algorithm
best_schedule = genetic_algorithm()
print(f"Best schedule: {best_schedule}")
pygame.quit()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


Best schedule: [15, 10, 1, 14, 4, 20, 17, 2, 7, 16, 13, 5, 11, 19, 18, 6, 3, 8, 12, 9]
